![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Science/DigitalDataCollection/digital-data-collection.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Digital Data Collection

We can collect data in this notebook using a micorcontroller device like a [micro:bit](https://www.microbit.org), [Arduino](https://www.arduino.cc), [Raspberry Pi Pico](https://www.raspberrypi.org/products/raspberry-pi-pico), or anything that can connect to your computer and send [serial data](https://en.wikipedia.org/wiki/Serial_communication).

Most of these devices have [onboard sensors or inputs](https://microbit.org/get-started/first-steps/sensors), and you can also connect [external sensors](https://pinouts.net/arduino-sensor-list). For this experiment we will be using the temperature sensor on a micro:bit.

## Setting Up

If you are not familiar with programming a micro:bit, follow the [first stesp](https://microbit.org/get-started/first-steps/set-up) instructions.



In [24]:
%%html
<body>
    <div class="buttons">
        <button id="connect" class="button">Connect</button>
        <button id="disconnect" disabled class="button">Disconnect</button>
        <button id="clearData" class="button">Clear</button>
        <button id="saveData" class="button">Save</button>
        <button id="generateDemoData" class="button">Generate Demo Data</button>
    </div>
    <p id="dataArea"></p>
</body>
<script type="text/javascript">
if (!('serial' in navigator)) {document.getElementById("dataArea").innerText="This only works in Chrome or Edge browsers";}
var decoder = new TextDecoder();
var connect = document.querySelector("#connect");
var disconnect = document.querySelector("#disconnect");
var port;
var reader;
var dataPoints;

async function readLoop() {
    while (true) {
        var {value, done} = await reader.read();
        if (value) {
            document.getElementById("dataArea").innerText += decoder.decode(value);
            dataPoints += decoder.decode(value)
        }
        if (done) {
            reader.releaseLock();
            break;
        }
    }
}

connect.addEventListener("click", async () => {
    port = await navigator.serial.requestPort();
    await port.open({baudRate: 115200});
    reader = port.readable.getReader();
    connect.disabled = true;
    disconnect.disabled = false;
    readLoop();
});

disconnect.addEventListener("click", async () => {
    await reader.cancel();
    await port.close();
    disconnect.disabled = true;
    connect.disabled = false;
});

clearData.addEventListener("click", async () => {
    document.getElementById("dataArea").innerText = "";
});

saveData.addEventListener("click", async () => {
    var dataFromDevice = "%%writefile data.csv" + "\n" + document.getElementById("dataArea").innerText;
    Jupyter.notebook.select_next().get_selected_cell().set_text(dataFromDevice);
    Jupyter.notebook.execute_cell();
    Jupyter.notebook.select_next();
    Jupyter.notebook.execute_cell();
    Jupyter.notebook.scroll_to_cell(Jupyter.notebook.get_selected_index(), 1000);
});

generateDemoData.addEventListener("click", async () => {
    document.getElementById("dataArea").innerText = "10756,16\n11262,17\n11769,17\n12275,17\n12782,17\n13289,16\n13796,16";
});

</script>

In [25]:
%%writefile data.csv
10756,16
11262,17
11769,17
12275,17
12782,17
13289,16
13796,16

Overwriting data.csv


In [26]:
import pandas as pd
df = pd.read_csv('data.csv', header=None)
df

,0,1
0,10756,16
1,11262,17
2,11769,17
3,12275,17
4,12782,17
5,13289,16
6,13796,16


You now have your data in a DataFrame called `df`. If there were any errors, you can go back to the cell that starts with `%%writefile data.csv` and change values, then run that cell and the `import pandas as pd` cell again.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)